[link text](https://)**Troll Classification with BERT using Keras and TensorFlow 2**

In [ ]:
!nvidia-smi

Sun Jul 10 23:23:00 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!pip install tensorflow-gpu

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 511.7 MB 6.5 kB/s 
     |████████████████████████████████| 438 kB 72.0 MB/s 
     |████████████████████████████████| 1.6 MB 54.0 MB/s 
     |████████████████████████████████| 5.8 MB 62.5 MB/s 
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.8.0
    Uninstalling tensorflow-estimator-2.8.0:
      Successfully uninstalled tensorflow-estimator-2.8.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.8.0
    Uninstalling tensorboard-2.8.0:
      Successfully uninstalled tensorboard-2.8.0
  Attempting uninstall: keras
    Found existing installation: keras 2.8.0
    Uninstalling keras-2.8.0:
      Successfully uninstalled keras-2.8.0
  Attempting uninstall: gast
    Found existing installation: gast 0.5.3
    Uninstalling gast-0.5.3:
      Successfully uninstalled gast-0.

In [ ]:
!pip install --upgrade grpcio

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.5 MB 35.7 MB/s 
  Attempting uninstall: grpcio
    Found existing installation: grpcio 1.46.3
    Uninstalling grpcio-1.46.3:
      Successfully uninstalled grpcio-1.46.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.8.2+zzzcolab20220527125636 requires keras<2.9,>=2.8.0rc0, but you have keras 2.9.0 which is incompatible.
tensorflow 2.8.2+zzzcolab20220527125636 requires tensorboard<2.9,>=2.8, but you have tensorboard 2.9.1 which is incompatible.
tensorflow 2.8.2+zzzcolab20220527125636 requires tensorflow-estimator<2.9,>=2.8, but you have tensorflow-estimator 2.9.0 which is incompatible.


In [ ]:
!pip install tqdm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install bert-for-tf2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 41 kB 114 kB/s 
  Created wheel for bert-for-tf2: filename=bert_for_tf2-0.14.9-py3-none-any.whl size=30535 sha256=6dc01f549223c97f040d018a760fa99a12bc510c6331a4fa9da679456b755158
  Stored in directory: /root/.cache/pip/wheels/47/b6/e5/8c76ec779f54bc5c2f1b57d2200bb9c77616da83873e8acb53
  Created wheel for params-flow: filename=params_flow-0.8.2-py3-none-any.whl size=19472 sha256=4e26df20950e4bfef3c9e29d9c51865a6f8c4b199bde0276eae4299db3ff0e55
  Stored in directory: /root/.cache/pip/wheels/0e/fc/d2/a44fff33af0f233d7def6e7de413006d57c10e10ad736fe8f5
  Created wheel for py-params: filename=py_params-0.10.2-py3-none-any.whl size=7911 sha256=f192e852c9b28b9305513afe5dbf381d0463d6b915a8c2bb5370c6080da6100c
  Stored in directory: /root/.cache/pip/wheels/e1/11/67/33cc51bbee127cb8fb2ba549cd29109b2f22da43ddf9969716
Successfully built bert-for-tf2 params-flow p

In [ ]:
!pip install sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.2 MB 34.7 MB/s 


In [ ]:
from bert import BertModelLayer
from bert.loader import StockBertConfig, map_stock_config_to_params, load_stock_weights
from bert.tokenization.bert_tokenization import FullTokenizer

In [ ]:
import os
import math
import datetime

from tqdm import tqdm

import pandas as pd
import numpy as np

import tensorflow as tf
from tensorflow import keras

import seaborn as sns
from pylab import rcParams
import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator
from matplotlib import rc

from sklearn.metrics import confusion_matrix, classification_report

%matplotlib inline
%config InlineBackend.figure_format='retina'

sns.set(style='whitegrid', palette='muted', font_scale=1.2)

HAPPY_COLORS_PALETTE = ["#01BEFE", "#FFDD00", "#FF7D00", "#FF006D", "#ADFF02", "#8F00FF"]

sns.set_palette(sns.color_palette(HAPPY_COLORS_PALETTE))

rcParams['figure.figsize'] = 12, 8

RANDOM_SEED = 42

np.random.seed(RANDOM_SEED)


#tf.random.set_seed(RANDOM_SEED)

# Data

In [ ]:
!sudo echo -ne '\n' | sudo add-apt-repository ppa:alessandro-strada/ppa >/dev/null 2>&1 # note: >/dev/null 2>&1 is used to supress printing
!sudo apt update >/dev/null 2>&1
!sudo apt install google-drive-ocamlfuse >/dev/null 2>&1
!google-drive-ocamlfuse
!sudo apt-get install w3m >/dev/null 2>&1 # to act as web browser 
!xdg-settings set default-web-browser w3m.desktop >/dev/null 2>&1 # to set default browser 
%cd /content
!mkdir gdrive
%cd gdrive
!mkdir "My Drive"
!google-drive-ocamlfuse "/content/gdrive/My Drive"

/usr/bin/xdg-open: 851: /usr/bin/xdg-open: www-browser: not found
/usr/bin/xdg-open: 851: /usr/bin/xdg-open: links2: not found
/usr/bin/xdg-open: 851: /usr/bin/xdg-open: elinks: not found
/usr/bin/xdg-open: 851: /usr/bin/xdg-open: links: not found
/usr/bin/xdg-open: 851: /usr/bin/xdg-open: lynx: not found
/usr/bin/xdg-open: 851: /usr/bin/xdg-open: w3m: not found
xdg-open: no method available for opening 'https://accounts.google.com/o/oauth2/auth?client_id=564921029129.apps.googleusercontent.com&redirect_uri=https%3A%2F%2Fgd-ocaml-auth.appspot.com%2Foauth2callback&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force&state=lhRZXn-YZPx0C1pRNg1uJ0-CRx0imwFo8llQ7-Xwc1U'
/bin/sh: 1: firefox: not found
/bin/sh: 1: google-chrome: not found
/bin/sh: 1: chromium-browser: not found
/bin/sh: 1: open: not found
Cannot retrieve auth tokens.
Failure("Error opening URL:https://accounts.google.com/o/oauth2/auth?client_id=564921029129.apps.go

In [ ]:
df = pd.read_csv("/content/Constraint_Train_p.csv", encoding = "ISO-8859-1")
df1= pd.read_csv("/content/Constraint_Val_p.csv", encoding = "ISO-8859-1")
#df= pd.read_csv("/content/Constraint_Test_p.csv", encoding = "ISO-8859-1")
#df= pd.read_csv("/content/GDrive/MyDrive/Njit/PhD/Research/DissertationProposal/BERT_Model_work/fakenews/covid_SiriTitle/fakedata_all_text.csv", encoding = "ISO-8859-1")

df=(df.append(df1)).sample(frac=1).reset_index(drop=True)
#df=(df).sample(frac=1).reset_index(drop=True)

df['text']=df['text'].astype(str)
df['label']=df['label'].astype(int)

In [ ]:
print(len(df))

8560


In [ ]:
train=df.head(6848)
test=df.tail(1712)

# Troll Classification with BERT

In [ ]:
!wget https://storage.googleapis.com/bert_models/2018_10_18/uncased_L-12_H-768_A-12.zip
!unzip uncased_L-12_H-768_A-12.zip

--2022-07-10 23:25:05--  https://storage.googleapis.com/bert_models/2018_10_18/uncased_L-12_H-768_A-12.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.200.128, 74.125.68.128, 142.251.12.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.200.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 407727028 (389M) [application/zip]
Saving to: ‘uncased_L-12_H-768_A-12.zip’

uncased_L-12_H-768_ 100%[===================>] 388.84M  57.2MB/s    in 8.4s    

2022-07-10 23:25:14 (46.2 MB/s) - ‘uncased_L-12_H-768_A-12.zip’ saved [407727028/407727028]

Archive:  uncased_L-12_H-768_A-12.zip
   creating: uncased_L-12_H-768_A-12/
  inflating: uncased_L-12_H-768_A-12/bert_model.ckpt.meta  
  inflating: uncased_L-12_H-768_A-12/bert_model.ckpt.data-00000-of-00001  
  inflating: uncased_L-12_H-768_A-12/vocab.txt  
  inflating: uncased_L-12_H-768_A-12/bert_model.ckpt.index  
  inflating: uncased_L-12_H-768_A-12/bert_config.json  


In [ ]:
os.makedirs("model", exist_ok=True)

In [ ]:
!mv uncased_L-12_H-768_A-12/ model

In [ ]:
bert_model_name="uncased_L-12_H-768_A-12"

bert_ckpt_dir = os.path.join("model/", bert_model_name)
bert_ckpt_file = os.path.join(bert_ckpt_dir, "bert_model.ckpt")
bert_config_file = os.path.join(bert_ckpt_dir, "bert_config.json")

## Preprocessing

In [ ]:
class TrollDetectionData:
  DATA_COLUMN = "text"
  LABEL_COLUMN = "label"

  def __init__(self, train, test, tokenizer: FullTokenizer, classes, max_seq_len=192):
    self.tokenizer = tokenizer
    self.max_seq_len = 0
    self.classes = classes
    
    ((self.train_x, self.train_y), (self.test_x, self.test_y)) = map(self._prepare, [train, test])

    print("max seq_len", self.max_seq_len)
    self.max_seq_len = min(self.max_seq_len, max_seq_len)
    self.train_x, self.test_x = map(self._pad, [self.train_x, self.test_x])

  def _prepare(self, df):
    x, y = [], []
    
    for _, row in tqdm(df.iterrows()):
      text, label = row[TrollDetectionData.DATA_COLUMN], row[TrollDetectionData.LABEL_COLUMN]
      tokens = self.tokenizer.tokenize(text)
      tokens = ["[CLS]"] + tokens + ["[SEP]"]
      token_ids = self.tokenizer.convert_tokens_to_ids(tokens)
      self.max_seq_len = max(self.max_seq_len, len(token_ids))
      x.append(token_ids)
      y.append(self.classes.index(label))

    return np.array(x), np.array(y)

  def _pad(self, ids):
    x = []
    for input_ids in ids:
      input_ids = input_ids[:min(len(input_ids), self.max_seq_len - 2)]
      input_ids = input_ids + [0] * (self.max_seq_len - len(input_ids))
      x.append(np.array(input_ids))
    return np.array(x)

In [ ]:
tokenizer = FullTokenizer(vocab_file=os.path.join(bert_ckpt_dir, "vocab.txt"))

In [ ]:
tokenizer.tokenize("I can't wait to visit Bulgaria again!")

['i', 'can', "'", 't', 'wait', 'to', 'visit', 'bulgaria', 'again', '!']

In [ ]:
tokens = tokenizer.tokenize("I can't wait to visit Bulgaria again!")
tokenizer.convert_tokens_to_ids(tokens)

[1045, 2064, 1005, 1056, 3524, 2000, 3942, 8063, 2153, 999]

## Training

In [ ]:
classes = train.label.unique().tolist()
data = TrollDetectionData(train, test, tokenizer, classes, max_seq_len=128)

6848it [00:04, 1614.75it/s]
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:28: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
1712it [00:01, 1616.61it/s]


max seq_len 1871


## Evaluation

In [ ]:
import tensorflow as tf
saved_model2 = tf.keras.models.load_model("/content/gdrive/My Drive/pretrainedModel_0506_fakenews/alenfirstBERTonColab")

In [ ]:
print(saved_model2)

In [ ]:
_, train_acc = saved_model2.evaluate(data.train_x, data.train_y)
_, test_acc = saved_model2.evaluate(data.test_x, data.test_y)

print("train acc", train_acc)
print("test acc", test_acc)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/util/dispatch.py:1082: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


54/54 [==============================] - 17s 289ms/step - loss: 0.0587 - acc: 0.9883
train acc 0.984375
test acc 0.9883177280426025


In [ ]:
##############################################################################################3

In [ ]:
import csv, numpy as np, pandas as pd, string, re
#df = pd.read_csv("/content/Constraint_Test_p.csv", encoding = "ISO-8859-1")
df = pd.read_csv("/content/AllTrue_7.10.csv", encoding = "ISO-8859-1")

In [ ]:
import string

sentences2 = (df["text"]).values.tolist()

final_sentence_list=[]

for st in sentences2:
  st=str(st)
  final_sentence_list.append(st)

print(len(final_sentence_list))

5001


In [ ]:
label_res=[]
i=0
textlist=[]
for _, row in tqdm(df.iterrows()):
  text = row[TrollDetectionData.DATA_COLUMN]

  final_sentence_list2=[]
  final_sentence_list2.append(text[:128])

  pred_tokens2 = map(tokenizer.tokenize, list(final_sentence_list2))
  pred_tokens2 = map(lambda tok: ["[CLS]"] + tok + ["[SEP]"], pred_tokens2)

  pred_token_ids2=[]
  pred_token_ids2 = list(map(tokenizer.convert_tokens_to_ids, pred_tokens2))
  
  max_seq_len = max(data.max_seq_len, len(pred_token_ids2))
  
  pred_token_ids2 = pred_token_ids2[:min(len(pred_token_ids2), max_seq_len - 2)]
  pred_token_ids2 = map(lambda tids: tids +[0]*(data.max_seq_len-len(tids)), pred_token_ids2)
  pred_token_ids2 = np.array(list(pred_token_ids2), dtype=object)

  predictions2=[]
  predictions2 = saved_model2.predict(pred_token_ids2.astype(np.float128)).argmax(axis=-1)

  print(i, final_sentence_list2)
  textlist.extend(final_sentence_list2)
  i+=1

  for label in predictions2:
    label_res.append(classes[label])

0it [00:00, ?it/s]

1/1 [==============================] - 0s 56ms/step


1it [00:00,  9.15it/s]

0 ['â\x9a¡Top Stories:Â\xa0Everything You Need to Know About #InformedConsent and more... @21stCenturyWire #Pfizer #Moderna - https://t.co']
1/1 [==============================] - 0s 54ms/step


2it [00:00,  9.13it/s]

...

4998 ['Tonight on @ActionNewsJax at 5: Iâ\x80\x99ll tell you about some differences between the #Moderna vax and theâ\x80¦ https://t.co/mMVputPu']
1/1 [==============================] - 0s 91ms/step


5000it [12:22,  5.41it/s]

4999 ['Vaccination time ð\x9f\x92\x89 \n\n#PfizerBioNTech https://t.co/vvcUoqyUys']
1/1 [==============================] - 0s 70ms/step


5001it [12:23,  6.73it/s]

5000 ['#PfizerBioNTech, @moderna_tx testing their #vaccines against new #coronavirus variant\nhttps://t.co/6zCkVgxAAr']


In [ ]:
print(len(textlist), len(label_res))

5000 5000


In [ ]:
from pandas import DataFrame

df2 = DataFrame (zip(textlist, label_res), columns=['text', 'label'])
df2.to_csv ("/content/AllTrue_7.10_Results.csv", index = None, header=True)

# References

- https://mccormickml.com/2019/07/22/BERT-fine-tuning/
- https://github.com/snipsco/nlu-benchmark/tree/master/2017-06-custom-intent-engines
- https://jalammar.github.io/illustrated-bert/
- https://towardsdatascience.com/bert-for-dummies-step-by-step-tutorial-fb90890ffe03
- https://www.reddit.com/r/MachineLearning/comments/ao23cp/p_how_to_use_bert_in_kaggle_competitions_a/